In [20]:
import json
import pandas as pd
import re
import nltk # pour stopword
from nltk.corpus import stopwords

ModuleNotFoundError: No module named 'nltk'

In [15]:
# Parser le fichier JSONL

data_input= pd.read_json('products.jsonl', lines=True)


In [ ]:
# ID produit (numéro après la tld)
def id_produit_url(data_input):
    pattern = r'(product|products)/((\d+))'
    data_input['id_product'] = data_input['url'].apply(lambda x : re.search(pattern,x).group(2) if re.search(pattern,x) else None)
    return data_input

id_produit_url(data_input)

# Variante (si présente)

def variant_url(data_input):
    pattern = r'variant=([^&]+)'
    data_input['variant'] = data_input['url'].apply(lambda x : re.search(pattern,x).group(1) if re.search(pattern,x) else None)
    return data_input

variant_url(data_input)

,url,title,description,product_features,links,product_reviews,id_product,variant
0,https://web-scraping.dev/products,web-scraping.dev product page 1,,{},"[https://web-scraping.dev/, https://web-scrapi...",[],None,None
1,https://web-scraping.dev/product/1,Box of Chocolate Candy,Indulge your sweet tooth with our Box of Choco...,"{'material': 'Premium quality chocolate', 'fla...","[https://web-scraping.dev/, https://web-scrapi...","[{'date': '2022-07-22', 'id': 'chocolate-candy...",1,None
2,https://web-scraping.dev/product/16,Red Energy Potion,"Elevate your game with our 'Red Potion', an ex...","{'flavor': 'Intense berry fusion', 'caffeine_c...","[https://web-scraping.dev/, https://web-scrapi...","[{'date': '2023-02-10', 'id': 'red-potion-1', ...",16,None
3,https://web-scraping.dev/product/10,Kids' Light-Up Sneakers,Make your child's every step magical with thes...,{'material': 'Breathable fabric upper with syn...,"[https://web-scraping.dev/, https://web-scrapi...","[{'date': '2022-07-01', 'id': 'kids-light-up-s...",10,None
4,https://web-scraping.dev/product/10?variant=bl...,Kids' Light-Up Sneakers,Make your child's every step magical with thes...,{'material': 'Breathable fabric upper with syn...,"[https://web-scraping.dev/, https://web-scrapi...","[{'date': '2022-07-01', 'id': 'kids-light-up-s...",10,blue-5
...,...,...,...,...,...,...,...,...
151,https://web-scraping.dev/products?page=1,web-scraping.dev product page 1,,{},"[https://web-scraping.dev/, https://web-scrapi...",[],None,None
152,https://web-scraping.dev/products?page=2,web-scraping.dev product page 2,,{},"[https://web-scraping.dev/, https://web-scrapi...",[],None,None
153,https://web-scraping.dev/products?page=3,web-scraping.dev product page 3,,{},"[https://web-scraping.dev/, https://web-scrapi...",[],None,None
154,https://web-scraping.dev/products?page=4,web-scraping.dev product page 4,,{},"[https://web-scraping.dev/, https://web-scrapi...",[],None,None


In [ ]:
# Utiliser la tokenization par espace | Supprimer les stopwords et la ponctuation
nltk.download("stopwords")

stopwords_en = stopwords.words("english")

def nettoyage(texte):
    token_propre=[]
    texte_tokens= texte.lower().split(' ')
    for tok in texte_tokens:
        tok = re.sub(r'[^a-zA-Z0-9]', '', tok)
        if tok not in stopwords_en:
            token_propre.append(tok)
    return token_propre
    

['web-scraping.dev', 'product', 'page', '1']


In [ ]:
# Créer un index inversé pour le titre | Pour chaque token, stocker uniquement la liste des urls des documents


def index_inverse_title(data_input):
    index_inv = {}
    for index, row in data_input.iterrows():
        title_tokens = nettoyage(row['title'])
        for token in title_tokens:
            if token not in index_inv:
                index_inv[token] = set()
            index_inv[token].add(row['links'])
    return index_inv

index_inverse = index_inverse_title(data_input)


# Créer le même type d'index pour la description

def index_invers_description(data_input):
    index_inv={}
    for index, row in data_input.iterrows():
        description_tokens = nettoyage(row['description'])
        for token in description_tokens:
            if token not in index_inv:
                index_inv[token] = set()
            index_inv[token].add(row['links'])
    return index_inv

In [29]:
# Créer un index pour les reviews 

from datetime import datetime

def parametres_reviews(data_input):
    for indx, row in data_input.iterrows():
        reviews=row['reviews']
        nb_total_reviews = len(reviews)
        if nb_total_reviews>0:
            s = 0
            for rev in reviews:
                s += int(rev['rating'])
            note_moyenne = s/nb_total_reviews
        else:
            note_moyenne=0
        

date_obj = datetime.strptime(data_input['product_reviews'][5][0]['date'], '%Y-%m-%d')
print(type(date_obj.date()))
print(data_input['product_reviews'][5][0]['date'])

<class 'datetime.date'>
2022-07-01
